# Visualisations for Dashboard

## Imports

In [1]:
from os import environ as ENV
from dotenv import load_dotenv

import pyodbc
import pandas as pd
import altair as alt
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [2]:
load_dotenv()
conn_str = (f"DRIVER={{{ENV['DB_DRIVER']}}};SERVER={ENV['DB_HOST']};"
            f"PORT={ENV['DB_PORT']};DATABASE={ENV['DB_NAME']};"
            f"UID={ENV['DB_USER']};PWD={ENV['DB_PASSWORD']};Encrypt=no;")

conn = pyodbc.connect(conn_str)

In [3]:
query = """
SELECT
    pr.*,
    p.plant_name,
    p.lat,
    p.lang,
    p.city_id,
    p.scientific_name,
    c.city_name,
    c.state_name
FROM
    gamma.plant_reading pr
LEFT JOIN
    gamma.plant p
ON pr.plant_id = p.plant_id
LEFT JOIN
    gamma.city c
ON p.city_id = c.city_id;
"""
recordings = pd.read_sql(query, conn)
conn.close()

recordings.set_index("reading_id")
recordings

/var/folders/wc/2kss00k17yg_nb8945wzxzv40000gp/T/ipykernel_96088/4289948099.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  recordings = pd.read_sql(query, conn)


,reading_id,plant_id,botanist_id,temperature,last_watered,soil_moisture,recording_taken,plant_name,lat,lang,city_id,scientific_name,city_name,state_name
0,1,1,1,14,2025-09-25 13:51:41,32.97,2025-09-26 09:28:06.098,Venus flytrap,43.74,-11.51,1,None,Stammside,Albania
1,2,2,2,14,2025-09-25 14:58:34,34.15,2025-09-26 09:28:04.689,Corpse flower,47.84,-48.71,2,None,Floshire,American Samoa
2,3,3,3,80,2025-09-25 13:58:19,35.61,2025-09-26 09:28:05.186,Rafflesia arnoldii,-25.49,-36.13,3,None,Dale City,Mozambique
3,4,4,4,16,2025-09-25 14:56:07,36.58,2025-09-26 09:28:06.053,Black bat flower,63.37,46.40,4,None,West Tedboro,Taiwan
4,5,5,5,16,2025-09-25 13:57:08,29.81,2025-09-26 09:28:05.664,Pitcher plant,82.89,0.63,5,'Sarracenia catesbaei',North Felicia,Saint Kitts and Nevis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14926,14927,41,42,18,2025-09-26 13:08:03,94.02,2025-09-26 14:51:05.391,Medinilla Magnifica,-34.19,-78.70,42,'Medinilla magnifica',Kathrynville,Macao
14927,14928,42,43,12,2025-09-26 14:50:57,99.99,2025-09-26 14:51:04.787,Calliandra Haematocephala,83.92,-148.55,43,'Calliandra haematocephala',Gavinstad,Jersey
14928,14929,43,44,15,2025-09-26 14:09:29,97.68,2025-09-26 14:51:05.524,Zamioculcas Zamiifolia,-82.18,-72.49,44,'Zamioculcas zamiifolia',Lake Norbertstead,Greenland
14929,14930,47,48,17,2025-09-26 14:08:03,97.58,2025-09-26 14:51:05.010,Crassula Ovata,15.80,148.99,48,'Crassula ovata',Ginaberg,Togo


## Live Dashboard

### Temperature over Time

In [4]:
temp_recordings = pd.concat(
    [recordings["plant_id_name"], pd.to_datetime(recordings["recording_taken"]), recordings["temperature"]], axis=1)
temp_recordings = temp_recordings.rename(columns={"recording_taken": "time"})
temp_recordings["time"] = temp_recordings["time"].dt.floor('min')
temp_recordings

KeyError: 'plant_id_name'

In [ ]:
alt.Chart(temp_recordings, title="Temperature over Time").mark_line().encode(
    x="time:T",
    y="temperature:Q",
    color="plant_name:N"
)

alt.Chart(...)

### Soil Moisture Level over Time

In [ ]:
soil_recordings = pd.concat(
    [recordings["plant_name"], pd.to_datetime(recordings["recording_taken"]), recordings["soil_moisture"]], axis=1)
soil_recordings = soil_recordings.rename(columns={"recording_taken": "time"})
soil_recordings["time"] = soil_recordings["time"].dt.floor('min')

soil_recordings.loc[soil_recordings["soil_moisture"]
                    < 0, "soil_moisture"] = None
soil_recordings

,plant_name,time,soil_moisture
0,Venus flytrap,2025-09-26 09:28:00,32.97
1,Corpse flower,2025-09-26 09:28:00,34.15
2,Rafflesia arnoldii,2025-09-26 09:28:00,35.61
3,Black bat flower,2025-09-26 09:28:00,36.58
4,Pitcher plant,2025-09-26 09:28:00,29.81
...,...,...,...
13309,Medinilla Magnifica,2025-09-26 14:16:00,96.23
13310,Calliandra Haematocephala,2025-09-26 14:16:00,NaN
13311,Zamioculcas Zamiifolia,2025-09-26 14:16:00,99.64
13312,Crassula Ovata,2025-09-26 14:16:00,99.52


In [ ]:
alt.Chart(soil_recordings, title="Soil Moisture over Time").mark_line().encode(
    x="time:T",
    y="soil_moisture:Q",
    color="plant_name:N"
)

alt.Chart(...)

### Plants with Low Moisture Levels

In [ ]:
filter_no_negative_recordings = soil_recordings["soil_moisture"] >= 0
latest_soil_recordings = soil_recordings[filter_no_negative_recordings]
latest_soil_recordings = latest_soil_recordings.sort_values(
    "time").groupby("plant_name").tail(1)
latest_soil_recordings = latest_soil_recordings.sort_values("soil_moisture")
latest_soil_recordings

,plant_name,time,soil_moisture
2294,Calliandra Haematocephala,2025-09-26 10:17:00,0.05
13298,Aglaonema Commutatum,2025-09-26 14:16:00,14.70
13307,Araucaria Heterophylla,2025-09-26 14:16:00,15.21
13313,Epipremnum Aureum,2025-09-26 14:16:00,15.31
13283,Manihot Esculenta ‘Variegata’,2025-09-26 14:16:00,15.87
13280,Cuphea ‘David Verity’,2025-09-26 14:16:00,16.75
13305,Caladium Bicolor,2025-09-26 14:16:00,16.93
13299,Monstera Deliciosa,2025-09-26 14:16:00,17.19
13291,Spathiphyllum,2025-09-26 14:16:00,18.09
13278,Canna ‘Striata’,2025-09-26 14:16:00,18.13


In [ ]:
latest_soil_recordings["low_moisture"] = latest_soil_recordings["soil_moisture"] <= 30

alt.Chart(latest_soil_recordings.head(30), title="Latest Plant Soil Moisture Levels").mark_bar().encode(
    x=alt.X("plant_name").sort('y'),
    y="soil_moisture",
    color=alt.Color("low_moisture", legend=None)
)

alt.Chart(...)